# Fine tune Stable Diffusion XL

The next notebook contains instructions to fine tune stable diffusion XL using your own images using OCI datascience service.

Before start go to the bucket "sdxl-bucket" and upload 15 images of 1024x1024 on the folder. "sdxl/input/".


1. First, execute the next cell to install a conda enviroment that would be used during the process.

In [ ]:
!odsc conda install -s onnx113_p39_cpu_v1

Make sure to go to the top of the notebook and select the python kernel  <span style="color:red">onnx113_p39_cpu_v1</span>

2. Load libraries and initialize variables.

In [ ]:
import ads
import oci
import os
import ocifs
from oci.object_storage import ObjectStorageClient
from datetime import datetime, timedelta

ads.set_auth(auth='resource_principal')
rps = oci.auth.signers.get_resource_principals_signer()

object_storage_client = ObjectStorageClient(config={}, signer=rps)
OBJECT_STORAGE_NAMESPACE = object_storage_client.get_namespace().data
fs = ocifs.OCIFileSystem()

os.environ['namespace'] = OBJECT_STORAGE_NAMESPACE
LOG_GROUP_ID = os.environ['loggroup_ocid']
LOG_ID = os.environ['log_ocid']
BUCKET_NAME = os.environ['bucket_name']

INPUT_FOLDER = "oci://{bucket}@{namespace}/sdxl/input/".format(bucket=BUCKET_NAME, namespace=OBJECT_STORAGE_NAMESPACE)
OUTPUT_FOLDER = "oci://{bucket}@{namespace}/sdxl/output/".format(bucket=BUCKET_NAME, namespace=OBJECT_STORAGE_NAMESPACE)

CONDA_ENV = "oci://{bucket}@{namespace}/conda_environments/cpu/ONNX 1.13 for CPU on Python 3.9/1.0/onnx113_p39_cpu_v1".format(bucket=BUCKET_NAME, namespace=OBJECT_STORAGE_NAMESPACE)


3. Prepare the conda enviroment for the training job. This will install dependecies and push the conda enviroment to a bucket.

In [ ]:
%%capture
if not os.path.exists("/home/datascience/job_artifact/kohya_ss"):
    !git clone https://github.com/bmaltais/kohya_ss.git /home/datascience/job_artifact/kohya_ss

if not fs.exists(CONDA_ENV):
    !conda init bash
    !source ~/.bashrc && conda activate /home/datascience/conda/onnx113_p39_cpu_v1 && cd /home/datascience/job_artifact/kohya_ss && bash setup.sh 
    !odsc conda init -b $bucket_name -n $namespace -a resource_principal
    !rm -rf /home/datascience/conda/onnx113_p39_cpu_v1/lib/python3.9/site-packages/library.pth
    !odsc conda publish -s onnx113_p39_cpu_v1 --force

!cp /home/datascience/repos/carlgira/oci-tf-odsc-sdxl/app/stable_main.py /home/datascience/job_artifact/

4. Next cell creates the job using the previous conda env and a python script to execute the lora training.

In [ ]:
from ads.jobs import Job, DataScienceJob, PythonRuntime

job = (
    Job(name="sdxl-train-job")
    .with_infrastructure(
        DataScienceJob()
        .with_log_group_id(LOG_GROUP_ID)
        .with_log_id(LOG_ID)
        .with_shape_name("VM.GPU2.1")
    )
    .with_runtime(
        PythonRuntime()
        .with_custom_conda(CONDA_ENV)
        .with_source("/home/datascience/job_artifact/")
        .with_entrypoint("stable_main.py")
        .with_working_dir("job_artifact")
        .with_environment_variable(full_input_folder=INPUT_FOLDER)
        .with_output("./output", OUTPUT_FOLDER)
    )
)

job.create()

5. Next cell will run the the job for stable diffusion fine tunning. This will take a while.

In [ ]:
job_run_env = job.run(
    name="Job Run - Passing dynamic values",
    env_var={'full_input_folder': INPUT_FOLDER}
)
job_run_watch = job_run_env.watch()

When the training ends, a file with the lora weights will be saved into "sdxl/output/sks.safetensors".

6. The next cell downloads ComfyUI to run it inside the notebook, to test the model for image inference.

In [ ]:
%%capture
import os

fs = ocifs.OCIFileSystem()

comfyui = "ComfyUI"
if not os.path.exists(comfyui):
    !git clone https://github.com/comfyanonymous/ComfyUI

if not os.path.exists("ComfyUI/models/loras/sks.safetensors"):
    fs.invalidate_cache(OUTPUT_FOLDER)
    fs.get(OUTPUT_FOLDER + "sks.safetensors", comfyui + "/models/loras/" , recursive=True, refresh=True)
    
if not os.path.exists("ComfyUI/models/checkpoints/sd_xl_base_1.0.safetensors"):
    !wget -O ComfyUI/models/checkpoints/sd_xl_base_1.0.safetensors https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors


if not os.path.exists("ComfyUI/venv"):
    ! cd ComfyUI && pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 xformers && pip install -r requirements.txt


7. Starts ComfyUI.

In [ ]:
import threading
import subprocess

def start_comfyui():
    subprocess.run(["python3", "ComfyUI/main.py"])

thread_1 = threading.Thread(target=start_comfyui)
thread_1.start()

8. Finally, generate images using your prompts. All the generated images are saved into "ComfyUI/output" if you want to download them.

In [ ]:
import inference
import matplotlib.pyplot as plt
img = inference.generate_image('portrait sks, pencil')
plt.axis('off')
plt.imshow(img)